In [1]:
import geopandas as gpd 
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt
import datetime as dt 
from shapely.geometry import Point, Polygon
import sys
import glob 
import os 
from scipy import stats 
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split 
from sklearn import preprocessing
import seaborn as sns
import copy

files_path = "C:/Users/Michael Ip/TDI/Capstone_Project/citibike_tripdata/2015_04_to_2015_05"
read_files = glob.glob(os.path.join(files_path,"*.csv"))

li = []

for file in read_files: 
    df = pd.read_csv(file, index_col = None, header = 0)
    li.append(df)


files_path_census = "C:/Users/Michael Ip/TDI/Capstone_Project/citibike_tripdata/Census_data/Census_data_combined_boroughs"
read_files_census = glob.glob(os.path.join(files_path_census,"*.csv"))

li = []

for file in read_files_census: 
    df_census = pd.read_csv(file, index_col = None, header = 0)
    new_header = df_census.iloc[0]
    df_census = df_census[1:]
    df_census.columns = new_header
    df_census.reset_index(inplace=True, drop=True)
    li.append(df_census)

df_census = pd.concat(li, axis = 0, ignore_index = True)
df_census = df_census.drop(df_census.index[[26, 65]])
df_census = df_census.drop(df_census[df_census['Total!!Estimate!!Total population'] == '0'].index)
df = df.drop(df[df['start station name'] == 'South St & Gouverneur Ln'].index)

#files_path_census = "C:/Users/Michael Ip/TDI/Capstone_Project/citibike_tripdata/Census_data/Census_data_2018-2012_MANHATTAN/ACSST5Y2014.S0101_data_with_overlays_2020-02-02T201411.csv"
#df_census = pd.read_csv(files_path_census)

#new_header = df_census.iloc[0]
#df_census = df_census[1:]
#df_census.columns = new_header
#df_census.reset_index(inplace=True, drop=True)



C:\Users\Michael Ip\Anaconda3\envs\geo_env\lib\site-packages\ipykernel_launcher.py:40: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [2]:
df_combined_census_total = pd.read_csv('df_combined_census_total.csv')
df_census_filtered_totals = pd.read_csv('df_census_filtered_totals.csv')

In [5]:
df_combined_census_total.shape

(965301, 50)

In [4]:
pd.options.display.max_columns = 52
df_combined_census_total.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,trip duration minutes,user type number,age,round trip,distance traveled,geometry,index_right,OBJECTID,postalCode,PO_NAME,STATE,borough,ST_FIPS,CTY_FIPS,BLDGpostalCode,Shape_Leng,Shape_Area,@id,population,Under 5 years,5 to 9 years,10 to 14 years,15 to 19 years,20 to 24 years,25 to 29 years,30 to 34 years,35 to 39 years,40 to 44 years,45 to 49 years,50 to 54 years,55 to 59 years,60 to 64 years,65 to 69 years,70 to 74 years,75 to 79 years,80 to 84 years,85 years and over
0,415,2015-05-01 00:00:11,2015-05-01 00:07:06,477,W 41 St & 8 Ave,40.756405,-73.990026,442,W 27 St & 7 Ave,40.746647,-73.993915,17012,Subscriber,1981.0,1,6.916667,1,34.0,no,1.133461,POINT (-73.9900262 40.75640548),109,110,10018,New York,NY,Manhattan,36,61,0,19509.953857,1.070580e+07,http://nyc.pediacities.com/Resource/PostalCode...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1523,2015-05-01 00:00:28,2015-05-01 00:25:52,293,Lafayette St & E 8 St,40.730287,-73.990765,324,DeKalb Ave & Hudson Ave,40.689888,-73.981013,17390,Customer,NaN,0,25.383333,0,NaN,no,4.566705,POINT (-73.9907647 40.73028666),121,122,10003,New York,NY,Manhattan,36,61,0,17396.136890,1.553838e+07,http://nyc.pediacities.com/Resource/PostalCode...,57068.0,1655.0,742.0,970.0,6163.0,6392.0,9131.0,6791.0,4337.0,3709.0,2625.0,3025.0,2910.0,2682.0,2340.0,1027.0,1141.0,742.0,742.0
2,642,2015-05-01 00:00:52,2015-05-01 00:11:35,380,W 4 St & 7 Ave S,40.734011,-74.002939,507,E 25 St & 2 Ave,40.739126,-73.979738,15003,Subscriber,1990.0,1,10.700000,1,25.0,no,2.035836,POINT (-74.00293877 40.73401143),120,121,10014,New York,NY,Manhattan,36,61,0,20454.072243,1.415105e+07,http://nyc.pediacities.com/Resource/PostalCode...,31835.0,955.0,764.0,700.0,541.0,1369.0,5539.0,4489.0,2961.0,2388.0,2228.0,2133.0,1719.0,1846.0,1624.0,1051.0,541.0,573.0,382.0
3,367,2015-05-01 00:00:48,2015-05-01 00:06:55,537,Lexington Ave & E 24 St,40.740259,-73.984092,280,E 10 St & 5 Ave,40.733320,-73.995101,14788,Subscriber,1978.0,1,6.116667,1,37.0,no,1.206541,POINT (-73.98409214 40.74025878),118,119,10010,New York,NY,Manhattan,36,61,0,23512.593499,9.768395e+06,http://nyc.pediacities.com/Resource/PostalCode...,30708.0,921.0,829.0,461.0,1351.0,3316.0,4852.0,3224.0,2733.0,2242.0,1996.0,1535.0,1935.0,1474.0,1259.0,798.0,768.0,553.0,491.0
4,2734,2015-05-01 00:00:51,2015-05-01 00:46:26,426,West St & Chambers St,40.717548,-74.013221,327,Vesey Pl & River Terrace,40.715338,-74.016584,21068,Subscriber,1956.0,2,45.566667,1,59.0,no,0.375153,POINT (-74.01322069 40.71754834),127,128,10013,New York,NY,Manhattan,36,61,0,19248.015538,1.558058e+07,http://nyc.pediacities.com/Resource/PostalCode...,26937.0,1535.0,1051.0,997.0,1077.0,2559.0,2801.0,2667.0,2020.0,2101.0,1589.0,1293.0,1455.0,2101.0,1293.0,781.0,539.0,404.0,620.0


In [3]:
df_census_filtered_totals

#np.where(pd.isnull(df_census_filtered_totals))

,postalCode,population,15 to 19 years,20 to 24 years,25 to 29 years,30 to 34 years,35 to 39 years,40 to 44 years,45 to 49 years,5 to 9 years,50 to 54 years,55 to 59 years,60 to 64 years,65 to 69 years,70 to 74 years,75 to 79 years,80 to 84 years,85 years and over
0,10451,47818,3730.0,4017.0,3634.0,3204.0,3013.0,3108.0,3060.0,4017.0,2965.0,2104.0,2487.0,1578.0,1339.0,1100.0,622.0,669.0
1,10452,75252,6246.0,6171.0,5343.0,5644.0,4967.0,5042.0,5268.0,6622.0,4590.0,3913.0,3236.0,2032.0,1580.0,903.0,602.0,452.0
2,10453,79002,6399.0,7347.0,6241.0,5925.0,4898.0,5372.0,5451.0,5925.0,5451.0,4424.0,3555.0,2212.0,1501.0,869.0,553.0,474.0
3,10454,38751,3565.0,3526.0,2868.0,2713.0,2868.0,2403.0,2558.0,3371.0,2054.0,1511.0,1124.0,775.0,969.0,698.0,620.0,271.0
4,10455,39951,2717.0,3436.0,3476.0,3156.0,2677.0,2517.0,2397.0,3476.0,2317.0,1838.0,1478.0,1119.0,1159.0,519.0,360.0,400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,10017,16472,181.0,1153.0,2981.0,2240.0,1334.0,1153.0,791.0,280.0,1219.0,972.0,692.0,988.0,725.0,511.0,280.0,280.0
168,10018,9168,211.0,1045.0,2411.0,1595.0,999.0,504.0,468.0,101.0,403.0,385.0,248.0,183.0,37.0,73.0,9.0,0.0
169,10280,9552,210.0,363.0,736.0,1395.0,1204.0,764.0,583.0,487.0,850.0,573.0,458.0,296.0,210.0,115.0,115.0,0.0
170,10282,5730,69.0,97.0,378.0,905.0,842.0,676.0,332.0,424.0,309.0,46.0,74.0,92.0,17.0,17.0,46.0,115.0


In [3]:
df_census_numcols = copy.deepcopy(df_census_filtered_totals)

In [3]:
df_combined_census_numcols = copy.deepcopy(df_combined_census_total)

In [4]:
df_combined_census_numcols = df_combined_census_numcols.rename(columns = {'15 to 19 years' : 17, 
                       '20 to 24 years' : 22, 
                       '25 to 29 years' : 27, 
                       '30 to 34 years' : 32, 
                       '35 to 39 years' : 37,
                       '40 to 44 years' : 42, 
                       '45 to 49 years' : 47, 
                       '50 to 54 years' : 52, 
                       '55 to 59 years' : 57, 
                       '60 to 64 years' : 62, 
                       '65 to 69 years' : 67, 
                       '70 to 74 years' : 72, 
                       '75 to 79 years' : 77, 
                       '80 to 84 years' : 82, 
                       '85 years and over' : 87})
#df_combined_census_numcols = df_combined_census_numcols.drop(columns = ['Under 5 years', '5 to 9 years', '10 to 14 years'])

In [6]:
df_combined_census_numcols.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,...,47,5 to 9 years,52,57,62,67,72,77,82,87
0,415,2015-05-01 00:00:11,2015-05-01 00:07:06,477,W 41 St & 8 Ave,40.756405,-73.990026,442,W 27 St & 7 Ave,40.746647,...,468.0,101.0,403.0,385.0,248.0,183.0,37.0,73.0,9.0,0.0
1,1523,2015-05-01 00:00:28,2015-05-01 00:25:52,293,Lafayette St & E 8 St,40.730287,-73.990765,324,DeKalb Ave & Hudson Ave,40.689888,...,2625.0,742.0,3025.0,2910.0,2682.0,2340.0,1027.0,1141.0,742.0,742.0
2,642,2015-05-01 00:00:52,2015-05-01 00:11:35,380,W 4 St & 7 Ave S,40.734011,-74.002939,507,E 25 St & 2 Ave,40.739126,...,2228.0,764.0,2133.0,1719.0,1846.0,1624.0,1051.0,541.0,573.0,382.0
3,367,2015-05-01 00:00:48,2015-05-01 00:06:55,537,Lexington Ave & E 24 St,40.740259,-73.984092,280,E 10 St & 5 Ave,40.733320,...,1996.0,829.0,1535.0,1935.0,1474.0,1259.0,798.0,768.0,553.0,491.0
4,2734,2015-05-01 00:00:51,2015-05-01 00:46:26,426,West St & Chambers St,40.717548,-74.013221,327,Vesey Pl & River Terrace,40.715338,...,1589.0,1051.0,1293.0,1455.0,2101.0,1293.0,781.0,539.0,404.0,620.0


In [5]:
#The following start stations were dropped because there were no proper zipcodes attached to them 

indexNames = df_combined_census_numcols[(df_combined_census_numcols['start station name'] == 'Central Park S & 6 Ave')| 
                                        (df_combined_census_numcols['start station name'] == 'Clinton Ave & Flushing Ave')|
                                        (df_combined_census_numcols['start station name'] == 'Sands St & Navy St')|
                                        (df_combined_census_numcols['start station name'] == 'Railroad Ave & Kay Ave')].index
df_combined_census_numcols.drop(indexNames, inplace=True)

In [6]:
start_stations = df_combined_census_numcols.groupby(df_combined_census_numcols['start station name'])

In [7]:
indiv_station_dict = dict()
for start_station, frame in start_stations: 
    indiv_station_dict[start_station] = pd.DataFrame(frame[['age', 17, 22, 27, 32, 37, 42, 47, 
                                                            52, 57, 62, 67, 72, 77, 82, 87]]).dropna()

In [10]:
df_combined_census_numcols[df_combined_census_numcols['start station name']=='1 Ave & E 44 St']

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,...,47,5 to 9 years,52,57,62,67,72,77,82,87
554,1311,2015-05-01 05:49:01,2015-05-01 06:10:52,455,1 Ave & E 44 St,40.75002,-73.969053,459,W 20 St & 11 Ave,40.746745,...,791.0,280.0,1219.0,972.0,692.0,988.0,725.0,511.0,280.0,280.0
1578,342,2015-05-01 07:16:19,2015-05-01 07:22:01,455,1 Ave & E 44 St,40.75002,-73.969053,519,Pershing Square North,40.751873,...,791.0,280.0,1219.0,972.0,692.0,988.0,725.0,511.0,280.0,280.0
2078,228,2015-05-01 07:38:16,2015-05-01 07:42:04,455,1 Ave & E 44 St,40.75002,-73.969053,359,E 47 St & Park Ave,40.755103,...,791.0,280.0,1219.0,972.0,692.0,988.0,725.0,511.0,280.0,280.0
2181,648,2015-05-01 07:42:10,2015-05-01 07:52:59,455,1 Ave & E 44 St,40.75002,-73.969053,325,E 19 St & 3 Ave,40.736245,...,791.0,280.0,1219.0,972.0,692.0,988.0,725.0,511.0,280.0,280.0
2907,1885,2015-05-01 08:05:12,2015-05-01 08:36:38,455,1 Ave & E 44 St,40.75002,-73.969053,315,South St & Gouverneur Ln,40.703554,...,791.0,280.0,1219.0,972.0,692.0,988.0,725.0,511.0,280.0,280.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959834,270,2015-05-31 15:08:47,2015-05-31 15:13:18,455,1 Ave & E 44 St,40.75002,-73.969053,385,E 55 St & 2 Ave,40.757973,...,791.0,280.0,1219.0,972.0,692.0,988.0,725.0,511.0,280.0,280.0
962579,571,2015-05-31 16:26:52,2015-05-31 16:36:23,455,1 Ave & E 44 St,40.75002,-73.969053,326,E 11 St & 1 Ave,40.729538,...,791.0,280.0,1219.0,972.0,692.0,988.0,725.0,511.0,280.0,280.0
963257,3841,2015-05-31 17:16:59,2015-05-31 18:21:01,455,1 Ave & E 44 St,40.75002,-73.969053,537,Lexington Ave & E 24 St,40.740259,...,791.0,280.0,1219.0,972.0,692.0,988.0,725.0,511.0,280.0,280.0
964051,722,2015-05-31 19:24:36,2015-05-31 19:36:39,455,1 Ave & E 44 St,40.75002,-73.969053,326,E 11 St & 1 Ave,40.729538,...,791.0,280.0,1219.0,972.0,692.0,988.0,725.0,511.0,280.0,280.0


In [10]:
bool_series = pd.isnull(df_combined_census_numcols[" population"])
df_combined_census_numcols[bool_series] 

df_combined_census_numcols[df_combined_census_numcols[' population'].isnull()]['start station name'].unique()

#np.where(pd.isnull(df_combined_census_numcols))

array([], dtype=object)

In [36]:
indiv_station_dict.keys()

dict_keys(['1 Ave & E 15 St', '1 Ave & E 18 St', '1 Ave & E 30 St', '1 Ave & E 44 St', '10 Ave & W 28 St', '11 Ave & W 27 St', '11 Ave & W 41 St', '11 Ave & W 59 St', '12 Ave & W 40 St', '2 Ave & E 31 St', '2 Ave & E 58 St', '3 Ave & Schermerhorn St', '5 Ave & E 29 St', '6 Ave & Broome St', '6 Ave & Canal St', '6 Ave & W 33 St', '8 Ave & W 31 St', '8 Ave & W 33 St', '8 Ave & W 52 St', '9 Ave & W 14 St', '9 Ave & W 16 St', '9 Ave & W 18 St', '9 Ave & W 22 St', '9 Ave & W 45 St', 'Adelphi St & Myrtle Ave', 'Allen St & E Houston St', 'Allen St & Hester St', 'Allen St & Rivington St', 'Ashland Pl & Hanson Pl', 'Atlantic Ave & Fort Greene Pl', 'Atlantic Ave & Furman St', 'Avenue D & E 12 St', 'Avenue D & E 3 St', 'Avenue D & E 8 St', 'Bank St & Hudson St', 'Bank St & Washington St', 'Barclay St & Church St', 'Barrow St & Hudson St', 'Bayard St & Baxter St', 'Bedford Ave & S 9th St', 'Bialystoker Pl & Delancey St', 'Bond St & Schermerhorn St', 'Broad St & Bridge St', 'Broadway & Battery Pl',

In [68]:
indiv_station_dict['1 Ave & E 44 St']

,age,17,22,27,32,37,42,47,52,57,62,67,72,77,82,87
554,64.0,181.0,1153.0,2981.0,2240.0,1334.0,1153.0,791.0,1219.0,972.0,692.0,988.0,725.0,511.0,280.0,280.0
1578,50.0,181.0,1153.0,2981.0,2240.0,1334.0,1153.0,791.0,1219.0,972.0,692.0,988.0,725.0,511.0,280.0,280.0
2078,34.0,181.0,1153.0,2981.0,2240.0,1334.0,1153.0,791.0,1219.0,972.0,692.0,988.0,725.0,511.0,280.0,280.0
2181,28.0,181.0,1153.0,2981.0,2240.0,1334.0,1153.0,791.0,1219.0,972.0,692.0,988.0,725.0,511.0,280.0,280.0
2907,38.0,181.0,1153.0,2981.0,2240.0,1334.0,1153.0,791.0,1219.0,972.0,692.0,988.0,725.0,511.0,280.0,280.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959834,43.0,181.0,1153.0,2981.0,2240.0,1334.0,1153.0,791.0,1219.0,972.0,692.0,988.0,725.0,511.0,280.0,280.0
962579,36.0,181.0,1153.0,2981.0,2240.0,1334.0,1153.0,791.0,1219.0,972.0,692.0,988.0,725.0,511.0,280.0,280.0
963257,44.0,181.0,1153.0,2981.0,2240.0,1334.0,1153.0,791.0,1219.0,972.0,692.0,988.0,725.0,511.0,280.0,280.0
964051,32.0,181.0,1153.0,2981.0,2240.0,1334.0,1153.0,791.0,1219.0,972.0,692.0,988.0,725.0,511.0,280.0,280.0


In [8]:
def custom_round(x, base=5):
    return int(base * round(float(x)/base))

test = indiv_station_dict['1 Ave & E 15 St']['age'].apply(lambda x: custom_round(x, base=5))

test.value_counts().max()


1089

In [12]:
indiv_station_dict['1 Ave & E 44 St'].loc[indiv_station_dict['1 Ave & E 44 St'].first_valid_index(), 
    [17, 22, 27, 32, 37, 42, 47, 52, 57, 62, 67, 72, 77, 82, 87]].max()

2981.0

In [37]:
import altair as alt 
alt.data_transformers.enable('default', max_rows=None)
vis = []

for station, frame in indiv_station_dict.items(): 
    chart = alt.Chart(pd.DataFrame(frame['age'])).mark_bar().encode(
        x=alt.X('age', bin=alt.Bin(extent=[18, 90], step=5), axis=alt.Axis(title='Age')), 
        y=alt.Y('count()', axis=alt.Axis(title='Counts'))).properties(
        width=400, height=250, title='{}'.format(station))
    
    max_biker_counts = frame['age'].apply(lambda x: custom_round(x, base=5)).value_counts().max()
    max_census_counts = frame.loc[frame.first_valid_index(), 
    [17, 22, 27, 32, 37, 42, 47, 52, 57, 62, 67, 72, 77, 82, 87]].max()
    ratio = max_census_counts/max_biker_counts
    
    data = frame.loc[
    frame.first_valid_index(), 
    [17, 22, 27, 32, 37, 42, 47, 52, 57, 62, 67, 72, 77, 82, 87]]
    data = pd.DataFrame(data)
    data = data.rename(columns={frame.first_valid_index():'value'})
    data.reset_index(level=0, inplace=True)
    data['value'] = data['value']/ratio 
    
    bar_filled = (alt.Chart(data).mark_area(color='#ff7f0e', interpolate='monotone', fillOpacity=0.3).encode(
    alt.X("index:Q", axis=alt.Axis(title='')),
    alt.Y("value:Q", axis=alt.Axis(title=''))))
          
    overlay = alt.layer(chart, bar_filled).configure_axis(
        labelFontSize=15, titleFontSize=20).configure_title(fontSize=20)
    vis.append(overlay.to_json())

In [50]:
import simplejson as json

json.loads(vis[0])

{'$schema': 'https://vega.github.io/schema/vega-lite/v4.0.2.json',
 'config': {'view': {'continuousHeight': 300, 'continuousWidth': 400}},
 'datasets': {'data-24c5bc0698d4b1fb95cfa76bcd7960b0': [{'age': 29.0},
   {'age': 24.0},
   {'age': 57.0},
   {'age': 31.0},
   {'age': 21.0},
   {'age': 28.0},
   {'age': 34.0},
   {'age': 27.0},
   {'age': 36.0},
   {'age': 28.0},
   {'age': 59.0},
   {'age': 35.0},
   {'age': 24.0},
   {'age': 30.0},
   {'age': 31.0},
   {'age': 29.0},
   {'age': 64.0},
   {'age': 41.0},
   {'age': 52.0},
   {'age': 39.0},
   {'age': 60.0},
   {'age': 48.0},
   {'age': 51.0},
   {'age': 26.0},
   {'age': 47.0},
   {'age': 23.0},
   {'age': 31.0},
   {'age': 56.0},
   {'age': 34.0},
   {'age': 25.0},
   {'age': 45.0},
   {'age': 33.0},
   {'age': 31.0},
   {'age': 32.0},
   {'age': 32.0},
   {'age': 25.0},
   {'age': 31.0},
   {'age': 45.0},
   {'age': 33.0},
   {'age': 32.0},
   {'age': 54.0},
   {'age': 24.0},
   {'age': 17.0},
   {'age': 28.0},
   {'age': 25.0}

In [44]:
indiv_station_dict['1 Ave & E 15 St']

,age,17,22,27,32
132,29.0,2410.0,6922.0,8715.0,6490.0
216,24.0,2410.0,6922.0,8715.0,6490.0
221,57.0,2410.0,6922.0,8715.0,6490.0
309,31.0,2410.0,6922.0,8715.0,6490.0
321,21.0,2410.0,6922.0,8715.0,6490.0
...,...,...,...,...,...
965085,20.0,2410.0,6922.0,8715.0,6490.0
965212,32.0,2410.0,6922.0,8715.0,6490.0
965222,27.0,2410.0,6922.0,8715.0,6490.0
965231,26.0,2410.0,6922.0,8715.0,6490.0


In [39]:
indiv_station_dict['1 Ave & E 15 St'].loc[
    indiv_station_dict['1 Ave & E 15 St'].first_valid_index(), 
    [17, 22, 27, 32]]

17    2410.0
22    6922.0
27    8715.0
32    6490.0
Name: 132, dtype: float64

In [68]:
indiv_station_dict['1 Ave & E 15 St']

,age,17,22,27,32
132,29.0,2410.0,6922.0,8715.0,6490.0
216,24.0,2410.0,6922.0,8715.0,6490.0
221,57.0,2410.0,6922.0,8715.0,6490.0
309,31.0,2410.0,6922.0,8715.0,6490.0
321,21.0,2410.0,6922.0,8715.0,6490.0
...,...,...,...,...,...
965085,20.0,2410.0,6922.0,8715.0,6490.0
965212,32.0,2410.0,6922.0,8715.0,6490.0
965222,27.0,2410.0,6922.0,8715.0,6490.0
965231,26.0,2410.0,6922.0,8715.0,6490.0


In [64]:
indiv_station_dict['1 Ave & E 15 St'].loc[132]

age      29.0
17     2410.0
22     6922.0
27     8715.0
32     6490.0
Name: 132, dtype: float64

In [10]:
df["age"] = 2015 - df["birth year"]

In [11]:
stations = df.take([4,5,6], axis=1)
#stations = df.take([4,5,6], axis=1).loc[['start station name']]
stations = stations.drop_duplicates().reset_index(drop=True)

age_bin = np.linspace(31, 44, 14)
groups = df.groupby(by = "start station name")["age"].agg(["mean"]).reset_index()
groups = groups.rename(columns = {"mean" : "mean age"})
groups["age bins"] = pd.cut(x = groups["mean age"], bins = age_bin)

stations = pd.merge(groups, stations, how='outer', on = 'start station name')


quantile_age_0 = stations["mean age"].quantile(0)
quantile_age_10 = stations["mean age"].quantile(0.10)
quantile_age_25 = stations["mean age"].quantile(0.25)
quantile_age_50 = stations["mean age"].quantile(0.50)
quantile_age_75 = stations["mean age"].quantile(0.75)
quantile_age_90 = stations["mean age"].quantile(0.90)
quantile_age_1 = stations["mean age"].quantile(1)

def colorfunc_age(row): 
    if row["mean age"] < quantile_age_25: 
        val = "red"
    elif (quantile_age_25 <= row["mean age"] < quantile_age_75):
        val = "darkred"
    else: 
        val = "black"
    return val 

stations['age bin color'] = stations.apply(colorfunc_age, axis=1)

In [12]:
station_list = stations.values.tolist()
print (station_list[69])
print (station_list[79])
print (station_list[244])
print (station_list[236])

['Central Park S & 6 Ave', 37.761320754716984, Interval(37.0, 38.0, closed='right'), 40.76590936, -73.97634151, 'darkred']
['Clinton Ave & Flushing Ave', 35.378571428571426, Interval(35.0, 36.0, closed='right'), 40.69794, -73.96986848, 'red']
['Sands St & Navy St', 37.392857142857146, Interval(37.0, 38.0, closed='right'), 40.699773, -73.979927, 'darkred']
['Railroad Ave & Kay Ave', 38.77777777777778, Interval(38.0, 39.0, closed='right'), 40.70531194, -73.97100056, 'darkred']


<font size = 5 >Be very careful of running the next cell.
    It is used to match the histogram to each station to 
    the tooltip popup of each station. Make sure you run the station_list
    cell  first before you run the following cell. Otherwise it will delete the     wrong thing 
<font>

In [13]:
station_list.pop(69)
station_list.pop(78)
station_list.pop(234)
station_list.pop(241)

['Sands St & Navy St',
 37.392857142857146,
 Interval(37.0, 38.0, closed='right'),
 40.699773,
 -73.979927,
 'darkred']

In [39]:
import folium 

m = folium.Map(location = [40.729135, -73.992330], 
               tiles = 'Stamen Terrain',
               zoom_start=12)
for station, histogram in zip(station_list, vis): 
    folium.CircleMarker([station[3],station[4]], 
                        radius=2,
                        fill=True, 
                        color=station[5], 
                        weight=4, 
                        tooltip='{}'.format(station[0]),
                        popup=folium.Popup(max_width=500).add_child(folium.VegaLite(histogram, width=500,height=250))).add_to(m)

#m.save("test_map.html")

In [40]:
legend_html =   '''
                <div style="position: fixed; 
                            bottom: 15px; left: 15px; width: 150px; height: 160px; 
                            border:2px solid grey; z-index:9999; font-size:14px;
                            background-color:rgba(255, 255, 255, 0.8);
                            "><b><font size='6'> &nbsp; Avg Age </font><b><br>
                              <b><font size='5'> &nbsp; 33 to 37 &nbsp; </font><b><i class="fa fa-circle aria-hidden="true" style="color:red" align="center"></i><br>
                              <b><font size='5'> &nbsp; 37 to 40 &nbsp </font><b><i class="fa fa-circle" style="color:darkred" align="center"></i><br>
                              <b><font size='5'> &nbsp; 40 to 44 &nbsp </font><b><i class="fa fa-circle" style="color:black" align="right"></i>
                </div>
                ''' 
m.get_root().html.add_child(folium.Element(legend_html))

In [41]:
m.save("test_map.html")

In [38]:
del m